In [1]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
df=pd.read_csv("/kaggle/input/bbc-full-text-document-classification/bbc_data.csv")

In [25]:
df.head()

,data,labels
0,Musicians to tackle US red tape Musicians gro...,entertainment
1,"U2s desire to be number one U2, who have won ...",entertainment
2,Rocker Doherty in on-stage fight Rock singer ...,entertainment
3,Snicket tops US box office chart The film ada...,entertainment
4,"Oceans Twelve raids box office Oceans Twelve,...",entertainment


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    2225 non-null   object
 1   labels  2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [27]:
df.describe()

,data,labels
count,2225,2225
unique,2126,5
top,Web radio takes Spanish rap global Spin the r...,sport
freq,2,511


In [28]:
df.duplicated().sum()

99

In [29]:
df.drop_duplicates(keep="first", inplace=True)
df

,data,labels
0,Musicians to tackle US red tape Musicians gro...,entertainment
1,"U2s desire to be number one U2, who have won ...",entertainment
2,Rocker Doherty in on-stage fight Rock singer ...,entertainment
3,Snicket tops US box office chart The film ada...,entertainment
4,"Oceans Twelve raids box office Oceans Twelve,...",entertainment
...,...,...
2220,Warning over Windows Word files Writing a Mic...,tech
2221,Fast lifts rise into record books Two high-sp...,tech
2222,Nintendo adds media playing to DS Nintendo is...,tech
2223,Fast moving phone viruses appear Security fir...,tech


In [30]:
stop_words = set(stopwords.words("english"))

In [31]:
lemma = WordNetLemmatizer()

In [32]:
def preprocessing(txt):
    tokens = word_tokenize(txt)
    
    words = [lemma.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]
#     for token in tokens:
#         if token.isalpha() and token not in stop_words:
#             words.append(lemma.lemmatize(token)) 
    
    return " ".join(words)

In [33]:
txt = df["data"][0]
txt

'Musicians to tackle US red tape  Musicians groups are to tackle US visa regulations which are blamed for hindering British acts chances of succeeding across the Atlantic.  A singer hoping to perform in the US can expect to pay $1,300 (xc2xa3680) simply for obtaining a visa. Groups including the Musicians Union are calling for an end to the "raw deal" faced by British performers. US acts are not faced with comparable expense and bureaucracy when visiting the UK for promotional purposes.  Nigel McCune from the Musicians Union said British musicians are "disadvantaged" compared to their US counterparts. A sponsor has to make a petition on their behalf, which is a form amounting to nearly 30 pages, while musicians face tougher regulations than athletes and journalists. "If you make a mistake on your form, you risk a five-year ban and thus the ability to further your career," says Mr McCune.  "The US is the worlds biggest music market, which means something has to be done about the creaky 

In [34]:
txt2 = preprocessing(txt)
txt2

'Musicians tackle US red tape Musicians group tackle US visa regulation blamed hindering British act chance succeeding across Atlantic A singer hoping perform US expect pay simply obtaining visa Groups including Musicians Union calling end raw deal faced British performer US act faced comparable expense bureaucracy visiting UK promotional purpose Nigel McCune Musicians Union said British musician disadvantaged compared US counterpart A sponsor make petition behalf form amounting nearly page musician face tougher regulation athlete journalist If make mistake form risk ban thus ability career say Mr McCune The US world biggest music market mean something done creaky bureaucracy say Mr McCune The current situation preventing British act maintaining momentum developing US added The Musicians Union stance endorsed Music Managers Forum MMF say British artist face uphill struggle succeed US thanks tough visa requirement also seen impractical The MMFs general secretary James Seller said Imagin

In [35]:
df["data"] = df["data"].apply(preprocessing)

In [36]:
encoder=LabelEncoder()
df['labels']=encoder.fit_transform(df['labels'])

In [37]:
X_train, X_test, y_train, y_test = train_test_split(df["data"], df["labels"], test_size=0.2, random_state=42)

In [38]:
vectorizer = TfidfVectorizer()

X_train_cv = vectorizer.fit_transform(X_train)
X_test_cv = vectorizer.transform(X_test)

In [43]:
from sklearn.linear_model import LogisticRegression

In [45]:
lr = LogisticRegression()

lr.fit(X_train_cv, y_train)
print(f"Train Score: {lr.score(X_train_cv, y_train)}\n")
y_pred = lr.predict(X_test_cv)
print(f"Test Score: {lr.score(X_test_cv, y_test)}\n")
print(f"Classification Report:\n {classification_report(y_test, y_pred)}\n")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred)}")

Train Score: 0.9958823529411764

Test Score: 0.9741784037558685

Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.94      0.96       108
           1       0.99      0.99      0.99        74
           2       0.95      0.99      0.97        80
           3       0.99      1.00      0.99        91
           4       0.96      0.97      0.97        73

    accuracy                           0.97       426
   macro avg       0.97      0.98      0.97       426
weighted avg       0.97      0.97      0.97       426


Confusion Matrix:
 [[101   0   3   1   3]
 [  0  73   1   0   0]
 [  0   1  79   0   0]
 [  0   0   0  91   0]
 [  2   0   0   0  71]]


In [53]:
df.to_csv('cleaned_BBC.csv', index=False)